In [1]:
import numpy as np
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier

import mean_variance

In [2]:
# Reads risk/return inputs and correlation matrix

return_risk_df = pd.read_excel('..\docs\optimization_assumptions_2023.xlsx',
                                sheet_name='return_risk', index_col=0)

corr_df = pd.read_excel('..\docs\optimization_assumptions_2023.xlsx',
                        sheet_name='corr', index_col=0)

expected_returns = return_risk_df.exp_return

In [3]:
# Produces covariance matrix from correlation matrix and standard deviation

cov_df = corr_df.copy()

for i in np.arange(return_risk_df.exp_risk.shape[0]):

    for j in np.arange(return_risk_df.exp_risk.shape[0]):

        cov_df.iloc[i,j] = cov_df.iloc[i,j] * return_risk_df.exp_risk[i] * return_risk_df.exp_risk[j]


In [7]:
#Runs mean variance portfolio algorithm given a maximum standard deviation
mean_variance.mv_portfolio(.09)

WEIGHTS
treasuries: 28.45
credit: 19.42
equity: 4.04
alt_assets: 6.47
alt_strats: 13.75
pvt_equity: 27.87

PERFORMANCE
Expected return: 8.36
Expected risk: 9.0
Expected Sharpe: 0.707








***TESTING***






In [11]:
ef = EfficientFrontier(expected_returns, cov_df, weight_bounds=(0,0.3))  # setup
clean_weights = ef.efficient_risk(0.12)  # find the portfolio that maximizes volatility
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

OrderedDict([('treasuries', 0.11514), ('credit', 0.3), ('equity', 0.27334), ('alt_assets', 0.01152), ('alt_strats', 0.0), ('pvt_equity', 0.3)])


In [19]:
ef.portfolio_performance(verbose=True)

Expected annual return: 9.2%
Annual volatility: 12.0%
Sharpe Ratio: 0.60


(0.09162798361962998, 0.1199999992934208, 0.5968998670115584)

In [20]:
sims = 1000     # Number of simulations to run
max_w = 0.3     # Maximum weight of an asset class 
ret_sd = 0.01   # Standard deviation that long term expected annual returns varies by
risk_sd = 0.01  # Standard deviation that long term expected annual volatility varies by

total_weights = []
total_performance = []

for sim in np.arange(sims):

    #Adds a layer of randomness to returns
    expected_returns = [er + ret_sd * np.random.randn() for er in return_risk_df.exp_return]

    #Adds a layer of randomness to risk
    expected_risk = [er + risk_sd * np.random.randn() for er in return_risk_df.exp_risk]

     # Produces covariance matrix from correlation matrix and standard deviation
    cov_df = corr_df.copy()
    
    for i in np.arange(return_risk_df.exp_risk.shape[0]):

        for j in np.arange(return_risk_df.exp_risk.shape[0]):

            cov_df.iloc[i,j] = cov_df.iloc[i,j] * return_risk_df.exp_risk[i] * return_risk_df.exp_risk[j]
    
    # Setup mean variance
    ef = EfficientFrontier(expected_returns, cov_df, weight_bounds=(0,max_w)) 
    clean_weights = ef.efficient_risk(0.12)  # find the portfolio that minimises volatility and L2_reg
    cleaned_weights = ef.clean_weights()

    # Saves each simulation's portfolios weights and performance 
    total_weights.append(list(cleaned_weights.values()))
    total_performance.append(list(ef.portfolio_performance(verbose=False)))



In [21]:
# Takes the average of the simulation weights for each asset class 
average_weights = np.mean(total_weights, axis=0)

# Prints the weights
for i in np.arange(average_weights.shape[0]):
    print(f'{corr_df.columns[i]}: {np.round(average_weights[i]*100, 2)}')

treasuries: 11.74
credit: 17.91
equity: 18.96
alt_assets: 12.99
alt_strats: 8.5
pvt_equity: 29.91


In [22]:
# Takes the average of the simulation performance for each metric
average_performance = np.mean(total_performance, axis=0)

# Prints the metrics
print(f'Expected return: {np.round(average_performance[0]*100,2)}')
print(f'Expected risk: {np.round(average_performance[1]*100,2)}')
print(f'Expected Sharpe: {np.round(average_performance[2],3)}')

Expected return: 9.41
Expected risk: 12.0
Expected Sharpe: 0.617
